In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [13]:
### We have to find a better way to work with this.
import sys
#sys.path.append('/Volumes/MacBookHD/Projects/PyPharmer_devel/') #Path for Angel in ashrey
#sys.path.append ('/home/angel/Documents/angel/projects/PyPharmer_devel/') #path for Angel in balmunt 
#sys.path.append('/home/diego/Trabajo/Proyectos/PyPharmer_devel/') # path for Diego
sys.path.append ('/Projects/PyPharmer_devel/') #path for angel in Groningen
import matplotlib as mt
from matplotlib import pylab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import numpy as np
import PyPharmer as phr
from PyPharmer import clique_analyzer as c_analyzer
import simplejson as json
import os 
from pprint import pprint
import mdtraj as md
from scipy.spatial import distance as dist

## Load files

In [3]:
json_file1='../Server_Files/ZINCpharmer.json'
json_file2='../Server_Files/pharmit.json'
json_file3='../Server_Files/gold_soln_l2199_3D_H_Ch_m1_4.json'
benchmark='../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1iep_B_STI.json'

## Saving points values using PyPharmer

In [83]:
data=phr.Descriptors (benchmark,select='enabled')

In [84]:
data.points

array([['Aromatic', 11.96, 61.004, 12.143, 1.1, 0, 0, 0],
       ['HydrogenDonor', 14.95, 55.73, 12.124, 0.5, 0, 0, 0],
       ['HydrogenDonor', 16.706, 51.802, 14.687, 0.5, 0, 0, 0],
       ['HydrogenAcceptor', 10.974, 60.526, 11.274, 0.5, 0, 0, 0],
       ['HydrogenAcceptor', 14.956, 52.572, 15.968, 0.5, 0, 0, 0],
       ['Hydrophobic', 11.96, 61.004, 12.143, 1, 0, 0, 0],
       ['Hydrophobic', 16.885, 53.668, 12.474, 1, 0, 0, 0],
       ['Hydrophobic', 17.235, 55.494, 10.11, 1, 0, 0, 0]], dtype=object)

# Save descriptors in a clique GRAPH

BCR_ABL_path= '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/'
ligands = []
for filename in os.listdir(BCR_ABL_path):
    if '.json' in filename:
        ligands.append(BCR_ABL_path+filename)
pprint (ligands)

G= nx.Graph (pharmacophore='descriptors')
for file in ligands:
    data=phr.Descriptors (input_file=file,select='enabled')
    for point  in data.points:
        node=(point[1],point[2],point[3],point[4])
        G.add_node (node_for_adding=node, name=point[0])
        result=list(it.combinations(G.nodes,2))
        for j in list(result):
            G.add_edge (j[0],j[1])

color_code={ 'Hydrophobic':        [0.100, 1.000, 0.000],\
             'HydrogenAcceptor':   [1.000, 0.84, 0.000],\
             'HydrogenDonor':      [1.000, 1.000, 1.000],\
             'Aromatic':           [0.627, 0.1254, 0.941],\
             'NegativeIon':        [1.00, 0.00, 0.00],\
             'PositiveIon':        [0.00, 0.00, 1.00],\
             'InclusionSphere':    [0.00, 1.00, 1.00],\
             'Other':              [0.74, 0.74, 0.74],\
             'PhenylalanineAnalog':[1.0, 1.0, 0.0],\
             'LeuValAnalog':       [1.0, 1.0, 0.0] \
             }

options = {'node_color': color_code[G.nodes[node]['name']],
           'node_size': 100,
           'width': 1,
           }


nx.draw_spectral (G,**options)
print ('Number or Nodes:', G.number_of_nodes(),' ','Number of Edges:',G.number_of_edges ())

# Adding the RMSD funtion between descriptors

In [85]:
data.points

array([['Aromatic', 11.96, 61.004, 12.143, 1.1, 0, 0, 0],
       ['HydrogenDonor', 14.95, 55.73, 12.124, 0.5, 0, 0, 0],
       ['HydrogenDonor', 16.706, 51.802, 14.687, 0.5, 0, 0, 0],
       ['HydrogenAcceptor', 10.974, 60.526, 11.274, 0.5, 0, 0, 0],
       ['HydrogenAcceptor', 14.956, 52.572, 15.968, 0.5, 0, 0, 0],
       ['Hydrophobic', 11.96, 61.004, 12.143, 1, 0, 0, 0],
       ['Hydrophobic', 16.885, 53.668, 12.474, 1, 0, 0, 0],
       ['Hydrophobic', 17.235, 55.494, 10.11, 1, 0, 0, 0]], dtype=object)

In [115]:
xyz=[]
for point in data.points:
    xyz.append ([point[1],point[2], point[3]])
xyz

[[11.781, 60.395, 12.354],
 [14.54, 55.006, 12.227],
 [15.995, 51.034, 14.819],
 [10.749, 59.946, 11.52],
 [14.232, 51.92, 16.039],
 [11.781, 60.395, 12.354],
 [16.366, 52.843, 12.597],
 [16.891, 54.608, 10.226]]

In [135]:
f=dist.cdist (xyz,xyz,'euclidean')
print (f)

[[ 0.          6.05553722 10.55757273  1.40077871  9.56097542  0.
   8.83821124  8.0081117 ]
 [ 6.05553722  0.          4.96107579  6.26698731  4.91422466  6.05553722
   2.85477582  3.1128132 ]
 [10.55757273  4.96107579  0.         10.85484505  2.31982004 10.55757273
   2.88918778  5.88828846]
 [ 1.40077871  6.26698731 10.85484505  0.          9.84730044  1.40077871
   9.11938742  8.2397114 ]
 [ 9.56097542  4.91422466  2.31982004  9.84730044  0.          9.56097542
   4.15370305  6.9344498 ]
 [ 0.          6.05553722 10.55757273  1.40077871  9.56097542  0.
   8.83821124  8.0081117 ]
 [ 8.83821124  2.85477582  2.88918778  9.11938742  4.15370305  8.83821124
   0.          3.00208111]
 [ 8.0081117   3.1128132   5.88828846  8.2397114   6.9344498   8.0081117
   3.00208111  0.        ]]


In [136]:
for i in range (len (f)):
    for j in range (len (f)):
        rmsd=((1/2)*((f[i][j]+f[i][j])**2))**(1/2)
        pprint (rmsd)

0.0
8.563822861316085
14.930662543906084
1.981000252397765
13.52126110982256
0.0
12.499118208897777
11.325180175167196
8.563822861316085
0.0
7.0160206670163126
8.862858455374312
6.949763161432193
8.563822861316085
4.037262686524122
4.402182640463704
14.930662543906084
7.0160206670163126
0.0
15.351069083291886
3.2807209573506877
14.930662543906084
4.085928535841033
8.327297400717713
1.981000252397765
8.862858455374312
15.351069083291886
0.0
13.926185838197044
1.981000252397765
12.896761376407637
11.65271161575708
13.52126110982256
6.949763161432193
3.2807209573506877
13.926185838197044
0.0
13.52126110982256
5.874223182685523
9.806792951826807
0.0
8.563822861316085
14.930662543906084
1.981000252397765
13.52126110982256
0.0
12.499118208897777
11.325180175167196
12.499118208897777
4.037262686524122
4.085928535841033
12.896761376407637
5.874223182685523
12.499118208897777
0.0
4.245583823221482
11.325180175167196
4.402182640463704
8.327297400717713
11.65271161575708
9.806792951826807
11.3251

In [137]:
BCR_ABL_path= '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/'
ligands = []
for filename in os.listdir(BCR_ABL_path):
    if '.json' in filename:
        ligands.append(BCR_ABL_path+filename)
pprint (ligands)

['../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1fpu_A_PRC.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1fpu_B_PRC.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1iep_A_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1iep_B_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1opj_A_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1opj_B_STI.json']


In [138]:
xyz=[]
for file in ligands:
    data=phr.Descriptors (input_file=file,select='enabled')
    for point  in data.points:
        xyz.append ((point[1],point[2], point[3]))
xyz

[(11.636, 60.16, 12.085),
 (14.657, 55.013, 12.152),
 (16.661, 51.228, 14.538),
 (10.615, 59.639, 11.334),
 (14.823, 51.786, 15.735),
 (11.636, 60.16, 12.085),
 (16.702, 53.106, 12.417),
 (16.107, 48.795, 17.282),
 (16.912, 54.912, 10.105),
 (11.454, 59.997, 12.107),
 (14.108, 54.572, 12.978),
 (16.006, 50.684, 15.318),
 (10.479, 59.572, 11.247),
 (13.797, 50.659, 15.803),
 (11.454, 59.997, 12.107),
 (16.069, 52.539, 13.146),
 (15.211, 48.225, 18.054),
 (16.333, 54.305, 10.797),
 (11.971, 60.92, 12.054),
 (14.852, 55.627, 12.128),
 (16.6, 51.81, 14.798),
 (10.961, 60.425, 11.216),
 (14.789, 52.612, 15.983),
 (11.971, 60.92, 12.054),
 (16.806, 53.618, 12.545),
 (17.148, 55.406, 10.154),
 (11.96, 61.004, 12.143),
 (14.95, 55.73, 12.124),
 (16.706, 51.802, 14.687),
 (10.974, 60.526, 11.274),
 (14.956, 52.572, 15.968),
 (11.96, 61.004, 12.143),
 (16.885, 53.668, 12.474),
 (17.235, 55.494, 10.11),
 (11.739, 60.045, 12.002),
 (14.39, 54.659, 12.103),
 (15.918, 50.842, 14.763),
 (10.694, 59.5

In [139]:
f= dist.cdist (xyz,xyz,'euclidean')
f

array([[ 0.        ,  5.96846203, 10.53795322, ...,  0.38550097,
         8.72774501,  7.86737631],
       [ 5.96846203,  0.        ,  4.90257453, ...,  6.10557974,
         2.79778591,  2.97729021],
       [10.53795322,  4.90257453,  0.        , ..., 10.61216967,
         2.5421902 ,  5.48367067],
       ...,
       [ 0.38550097,  6.10557974, 10.61216967, ...,  0.        ,
         8.83821124,  8.0081117 ],
       [ 8.72774501,  2.79778591,  2.5421902 , ...,  8.83821124,
         0.        ,  3.00208111],
       [ 7.86737631,  2.97729021,  5.48367067, ...,  8.0081117 ,
         3.00208111,  0.        ]])

In [141]:
for i in range (len (f)):
    for j in range (len (f)):
        rmsd=((1/2)*((f[i][j]+f[i][j])**2))**(1/2)
        if rmsd > 0.5:
            print ('diferent point', rmsd)
        if rmsd < 0.5:
            print ('same point', rmsd)

same point 0.0
diferent points 8.440679948914067
diferent points 14.902916358887609
diferent points 1.937979876056505
diferent points 13.682349578928314
same point 0.0
diferent points 12.290931290996623
diferent points 18.770289022814747
diferent points 10.890186407954637
same point 0.34692074022750025
diferent points 8.73315258082669
diferent points 15.449515526384637
diferent points 2.1847823690244272
diferent points 14.748725097444861
same point 0.34692074022750025
diferent points 12.5584116033836
diferent points 19.537339174002167
diferent points 10.7704946961595
diferent points 1.1754029096441851
diferent points 7.860355462700138
diferent points 14.263531470151417
diferent points 1.6006317502786216
diferent points 12.814688213140414
diferent points 1.1754029096441851
diferent points 11.810018120223182
diferent points 10.650016056325923
diferent points 1.28115260605441
diferent points 7.824195421894829
diferent points 14.30604543540946
diferent points 1.5683883447666949
diferent po

diferent points 14.826865548726067
diferent points 1.6909251905391902
diferent points 13.559724628472363
same point 0.34692074022750025
diferent points 12.25752707523014
diferent points 18.650246003739465
diferent points 10.922911058870707
same point 0.0
diferent points 8.6293663730311
diferent points 15.346868996639026
diferent points 1.9343474351832453
diferent points 14.584122119620366
same point 0.0
diferent points 12.489956765337498
diferent points 19.39390842506997
diferent points 10.762862537447925
diferent points 1.498016688825597
diferent points 7.828632703097007
diferent points 14.195053081971901
diferent points 1.5621229144980864
diferent points 12.703056797479887
diferent points 1.498016688825597
diferent points 11.792148998380233
diferent points 10.706365022732973
diferent points 1.594604026082962
diferent points 7.801229903034522
diferent points 14.240641067030655
diferent points 1.5518569521705308
diferent points 12.829805142713584
diferent points 1.594604026082962
difer

diferent points 4.405717648692436
diferent points 12.332675541017045
diferent points 5.910095261499598
diferent points 11.810018120223182
diferent points 0.7607154527154031
diferent points 9.611357760483166
diferent points 3.9087778140999494
diferent points 11.792148998380233
diferent points 4.093118371119996
diferent points 5.820289511699573
diferent points 12.423030950617488
diferent points 7.539804506749499
diferent points 11.792148998380233
diferent points 2.0340162241240876
diferent points 11.133512922703238
diferent points 2.739051660703024
diferent points 12.40463703620545
diferent points 4.00701534811136
diferent points 4.095682849049716
diferent points 12.826964956684021
diferent points 5.81380581031049
diferent points 12.40463703620545
same point 0.0
diferent points 4.249891528027507
diferent points 12.505863904584915
diferent points 4.020577321728805
diferent points 3.9739199790634956
diferent points 12.911090503904001
diferent points 5.697989996481217
diferent points 12.505

diferent points 8.22900334183916
diferent points 14.723917209764528
diferent points 1.936200402850904
diferent points 13.539381522063701
same point 0.2478830369347575
diferent points 12.079181677580646
diferent points 18.62902702773282
diferent points 10.649528346363514
same point 0.4348654964468898
diferent points 8.546164753853038
diferent points 15.286037158138802
diferent points 2.182362939568029
diferent points 14.613689540974933
same point 0.4348654964468898
diferent points 12.361041380077973
diferent points 19.411022023582372
diferent points 10.536077163726546
diferent points 1.2823049559289705
diferent points 7.645308234466415
diferent points 14.089866003621184
diferent points 1.6537714473288012
diferent points 12.68072158830088
diferent points 1.2823049559289705
diferent points 11.599626459502911
diferent points 10.410831474959146
diferent points 1.4059893313962184
diferent points 7.608499194979261
diferent points 14.12992307126971
diferent points 1.6410789133981367
diferent p

# All together

In [142]:
BCR_ABL_path= '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/'
ligands = []
for filename in os.listdir(BCR_ABL_path):
    if '.json' in filename:
        ligands.append(BCR_ABL_path+filename)
pprint (ligands)

['../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1fpu_A_PRC.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1fpu_B_PRC.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1iep_A_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1iep_B_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1opj_A_STI.json',
 '../benchmarks/BCR-ABL_Tyrosin_Kinase_Inhibitors/1opj_B_STI.json']


In [171]:
xyz=[]
same_point=[]
dif_point=[]
for file in ligands:
    data=phr.Descriptors (input_file=file,select='enabled')
    for point  in data.points:
        xyz.append ((point[1],point[2], point[3]))
        f= dist.cdist (xyz,xyz,'euclidean')
        for i in range (len (f)):
            for j in range (len (f)):
                rmsd=((1/2)*((f[i][j]+f[i][j])**2))**(1/2)
                if rmsd >= 1:
                    dif_point.append ((point))
                if rmsd < 1:
                    same_point.append ((point))

In [172]:
print ('diferent points',len (dif_point))
dif_point

diferent points 39476


[array(['HydrogenDonor', 14.657, 55.013, 12.152, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 14.657, 55.013, 12.152, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 1

In [173]:
print ('same points',len (same_point))
same_point

same points 3449


[array(['Aromatic', 11.636, 60.16, 12.085, 1.1, 0, 0, 0], dtype=object),
 array(['HydrogenDonor', 14.657, 55.013, 12.152, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 14.657, 55.013, 12.152, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenDonor', 16.661, 51.228, 14.538, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 10.615, 59.639, 11.334, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 14.823, 51.786, 15.735, 0.5, 0, 0, 0],
       dtype=object),
 array(['HydrogenAcceptor', 14.823, 51.786, 15.735, 

### Merging repeated points

In [182]:
for element in same_point:
    if element.any is not element.any:
        print (element)

['Aromatic' 11.636 60.16 12.085 1.1 0 0 0]
['HydrogenDonor' 14.657 55.013 12.152 0.5 0 0 0]
['HydrogenDonor' 14.657 55.013 12.152 0.5 0 0 0]
['HydrogenDonor' 16.661 51.228 14.538 0.5 0 0 0]
['HydrogenDonor' 16.661 51.228 14.538 0.5 0 0 0]
['HydrogenDonor' 16.661 51.228 14.538 0.5 0 0 0]
['HydrogenAcceptor' 10.615 59.639 11.334 0.5 0 0 0]
['HydrogenAcceptor' 10.615 59.639 11.334 0.5 0 0 0]
['HydrogenAcceptor' 10.615 59.639 11.334 0.5 0 0 0]
['HydrogenAcceptor' 10.615 59.639 11.334 0.5 0 0 0]
['HydrogenAcceptor' 14.823 51.786 15.735 0.5 0 0 0]
['HydrogenAcceptor' 14.823 51.786 15.735 0.5 0 0 0]
['HydrogenAcceptor' 14.823 51.786 15.735 0.5 0 0 0]
['HydrogenAcceptor' 14.823 51.786 15.735 0.5 0 0 0]
['HydrogenAcceptor' 14.823 51.786 15.735 0.5 0 0 0]
['Hydrophobic' 11.636 60.16 12.085 1 0 0 0]
['Hydrophobic' 11.636 60.16 12.085 1 0 0 0]
['Hydrophobic' 11.636 60.16 12.085 1 0 0 0]
['Hydrophobic' 11.636 60.16 12.085 1 0 0 0]
['Hydrophobic' 11.636 60.16 12.085 1 0 0 0]
['Hydrophobic' 11.636 60

['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.918 50.842 14.763 0.5 0 0 0]
['HydrogenDonor' 15.

['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.995 51.034 14.819 0.5 0 0 0]
['HydrogenDonor' 15.

['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydrophobic' 16.891 54.608 10.226 1 0 0 0]
['Hydropho